In [1]:
import re

In [2]:
import csv

In [3]:
import math

In [4]:
# calculate fitness and sigma values for each single mutant, which is exported to a single mutants file
# calculate fitness and sigma values for each double mutant, which is exported to a double mutants file
def counts_to_fitness(WT1, WT2):
    #open file with the name 'fitness-double library name.csv'
    countsfile= '12bp-flank-counts-' + WT1 + 'X-' + WT2 + 'X.csv'
    WT_dbl= WT1[0] + ':' + WT2[0]        
    with open('Singles-A3-processed.txt', 'a') as singleA3outfile:
        #put counts into dNS, dA3, and dC2 libraries
        dNS={}
        dA3={}
        dC2={}
        with open(countsfile) as infile:
            reader=csv.reader(infile)
            dNS={rows[0]:rows[1] for rows in reader}
            # deletes the title row and then converts string values from dictionary into integers for #MATHS later on.
            dNS.pop('AA')
            dNS={k:int(v) for k, v in dNS.items()}
        with open(countsfile) as infile:
            reader=csv.reader(infile)
            dA3={rows[0]:rows[2] for rows in reader}
            dA3.pop('AA')
            dA3={k:int(v) for k, v in dA3.items()}
        with open(countsfile) as infile:
            reader=csv.reader(infile)
            dC2={rows[0]:rows[3] for rows in reader}
            dC2.pop('AA')
            dC2={k:int(v) for k, v in dC2.items()}

        # A3 calculates fitness of each double AA mutation based on frequency of mut vs wt in selected vs naive conditions
        fitnessA3={}
        for k,v in dNS.items():
            for k2,v2 in dA3.items():
                if k2 in k:
                    wt_sel= dA3[WT_dbl]/sum(dA3.values())
                    wt_ns= dNS[WT_dbl]/sum(dNS.values())
                    a_sel= v2/sum(dA3.values())
                    a_ns= v/sum(dNS.values())
                    Fu= math.log10(a_sel/a_ns)-math.log10(wt_sel/wt_ns)
                    fitnessA3[k]= Fu
        #pull WT fitness from double position library 
        wt_fitnessA3={}
        for k,v in fitnessA3.items():
            if k== WT_dbl:
                wt_fitnessA3[k]= v
        #remove wildtype fitness from double position library
        for k1,v1 in sorted(wt_fitnessA3.items()):
            for k2,v2 in sorted(fitnessA3.items()):
                if k1 in k2:
                    fitnessA3.pop(k1)

        # pull single mutant fitness (positions 1 and 2) from double position library
        single_mutsA3={}
        for k,v in fitnessA3.items():
            if str(k[-1])== WT2[0]:
                single_mutsA3[k]= v
            if str(k[0])== WT1[0]:
                single_mutsA3[k]= v     
        #remove A3 single mutant fitness from double position library
        for k1,v1 in sorted(single_mutsA3.items()):
            for k2,v2 in sorted(fitnessA3.items()):
                    if k1 in k2:
                        fitnessA3.pop(k1)                    
        #calculate sigma for A3 WT based on counts
        wt_sigmaA3={}
        for k,v in wt_fitnessA3.items():
            sigma= math.sqrt(1/dNS[k] + 1/dA3[k])
            wt_sigmaA3[k]=sigma
        
        #calculate sigma for A3 single mutants based on counts
        singles_sigmaA3={}
        for k,v in single_mutsA3.items():
            sigma= math.sqrt(1/dNS[k] + 1/dNS[WT_dbl] + 1/dA3[k] + 1/dA3[WT_dbl])
            singles_sigmaA3[k]=sigma

        #calculate sigma for A3 double mutants based on counts
        doubles_sigmaA3={}
        for k,v in fitnessA3.items():
            sigma=math.sqrt(1/dNS[k] + 1/dNS[WT_dbl] + 1/dA3[k] + 1/dA3[WT_dbl])
            doubles_sigmaA3[k]=sigma

        # print A3 singles fitness/sigma/etc. to a single A3 outfile
        # if naive library has at least 50 counts, and post-selection library has at least 2 counts
        print('WT_AA\tPos\tMut\tcount_r1_t0\tcount_r1_t1\tfitness\tsigma\tis.fitness\tis.reads0', file=singleA3outfile)
        for k,v in sorted(single_mutsA3.items()):
            for k2,v2 in sorted(singles_sigmaA3.items()):
                if k in k2:
                    if k2[0]== WT1[0]:
                        print(str(k2[0]) + '\t' +  str(WT1[1:]) + '\t' + str(k2[-1]) + '\t' + str(dNS[k2]) + '\t' + str(dA3[k2])
                              + '\t' + str(v) + '\t' + str(v2) + '\tTRUE', file=singleA3outfile, end='')
                        if dNS[k2]>=50 and dA3[k2]>=2:
                            print('\tTRUE', file=singleA3outfile)
                        else:
                            print('\tFALSE', file=singleA3outfile)
                    if k2[-1]== WT2[0]:
                        print(str(k2[0]) + '\t' +  str(WT2[1:]) + '\t' + str(k2[-1]) + '\t' + str(dNS[k2]) + '\t' + str(dA3[k2])
                              + '\t' + str(v) + '\t' + str(v2) + '\tTRUE', file=singleA3outfile, end='')
                        if dNS[k2]>=50 and dA3[k2]>=2:
                            print('\tTRUE', file=singleA3outfile)
                        else:
                            print('\tFALSE', file=singleA3outfile)
        
        # print A3 doubles to a single A3 outfile
        # if naive library has at least 50 counts, and post-selection library has at least 2 counts
        with open('Doubles-A3-processed.txt', 'a') as doubleA3outfile:
            print('Pos1\tPos2\tMut1\tMut2\tWT_AA1\tWT_AA2\tcount_r1_t0\tcount_r1_t1\tis.fitness\tis.reads0\tfitness1\tfitness2\tsigma1\tsigma2\tfitness\tsigma', file=doubleA3outfile)
            for k,v in sorted(fitnessA3.items()):
                for k2,v2 in sorted(doubles_sigmaA3.items()):
                    if k in k2:
                        print(str(WT1[1:]) + '\t' + str(WT2[1:]) + '\t' + str(k[0]) + '\t' + str(k[-1]) + '\t'
                             + str(WT1[0]) + '\t' + str(WT2[0]) + '\t' + str(dNS[k]) + '\t' + str(dA3[k]) + '\t'
                             + 'TRUE', file=doubleA3outfile, end='')
                        if dNS[k]>=50 and dA3[k]>=2:
                            print('\tTRUE', file=doubleA3outfile, end='')
                        else:
                            print('\tFALSE', file=doubleA3outfile, end='')
                        print('\t' + str(single_mutsA3[str(k[0] + ':' + WT2[0])]) + '\t' + str(single_mutsA3[str(WT1[0] + ':' + k[-1])])+ '\t'
                              + str(singles_sigmaA3[str(k[0] + ':' + WT2[0])]) + '\t' + str(singles_sigmaA3[str(WT1[0] + ':' + k[-1])]) + '\t'
                              + str(v) + '\t' + str(doubles_sigmaA3[str(k)]), file=doubleA3outfile)
        
        # calculate C2 fitness with counts that were already put into dC2 above
        with open('Singles-C2-processed.txt', 'a') as singleC2outfile:
            # C2 calculates fitness of each double AA mutation based on frequency of mut vs wt in selected vs naive conditions!!!!!!
            fitnessC2={}
            for k,v in dNS.items():
                for k2,v2 in dC2.items():
                    if k2 in k:
                        wt_sel= dC2[WT_dbl]/sum(dC2.values())
                        wt_ns= dNS[WT_dbl]/sum(dNS.values())
                        a_sel= v2/sum(dC2.values())
                        a_ns= v/sum(dNS.values())
                        Fu= math.log10(a_sel/a_ns)-math.log10(wt_sel/wt_ns)
                        fitnessC2[k]= Fu
            
            #pull WT fitness from C2 double position library 
            wt_fitnessC2={}
            for k,v in fitnessC2.items():
                if k== WT_dbl:
                    wt_fitnessC2[k]= v
            
            #remove wildtype fitness from C2 double position library
            for k1,v1 in sorted(wt_fitnessC2.items()):
                for k2,v2 in sorted(fitnessC2.items()):
                    if k1 in k2:
                        fitnessC2.pop(k1)

            #pull C2 single mutant fitness (positions 1 and 2) from double position library
            single_mutsC2={}
            for k,v in fitnessC2.items():
                if str(k[-1])== WT2[0]:
                    single_mutsC2[k]= v
                if str(k[0])== WT1[0]:
                    single_mutsC2[k]= v     
           
            #remove C2 single mutant fitness from double position library
            for k1,v1 in sorted(single_mutsC2.items()):
                for k2,v2 in sorted(fitnessC2.items()):
                        if k1 in k2:
                            fitnessC2.pop(k1)                    
            
            #calculate C2 sigma based on counts
            wt_sigmaC2={}
            for k,v in wt_fitnessC2.items():
                sigma= math.sqrt(1/dNS[k] + 1/dC2[k])
                wt_sigmaC2[k]=sigma

            singles_sigmaC2={}
            for k,v in single_mutsC2.items():
                sigma= math.sqrt(1/dNS[k] + 1/dNS[WT_dbl] + 1/dC2[k] + 1/dC2[WT_dbl])
                singles_sigmaC2[k]=sigma

            doubles_sigmaC2={}
            for k,v in fitnessC2.items():
                sigma=math.sqrt(1/dNS[k] + 1/dNS[WT_dbl] + 1/dC2[k] + 1/dC2[WT_dbl])
                doubles_sigmaC2[k]=sigma

            # print C2 singles to a single C2 outfile
            # if naive library has at least 50 counts, and post-selection library has at least 2 counts
            print('WT_AA\tPos\tMut\tcount_r1_t0\tcount_r1_t1\tfitness\tsigma\tis.fitness\tis.reads0', file=singleC2outfile)
            for k,v in sorted(single_mutsC2.items()):
                for k2,v2 in sorted(singles_sigmaC2.items()):
                    if k in k2:
                        if k2[0]== WT1[0]:
                            print(str(k2[0]) + '\t' +  str(WT1[1:]) + '\t' + str(k2[-1]) + '\t' + str(dNS[k2]) + '\t' + str(dC2[k2])
                                  + '\t' + str(v) + '\t' + str(v2) + '\tTRUE', file=singleC2outfile, end='')
                            if dNS[k2]>=50 and dC2[k2]>=2:
                                print('\tTRUE', file=singleC2outfile)
                            else:
                                print('\tFALSE', file=singleC2outfile)
                        if k2[-1]== WT2[0]:
                            print(str(k2[0]) + '\t' +  str(WT2[1:]) + '\t' + str(k2[-1]) + '\t' + str(dNS[k2]) + '\t' + str(dC2[k2])
                                  + '\t' + str(v) + '\t' + str(v2) + '\tTRUE', file=singleC2outfile, end='')
                            if dNS[k2]>=50 and dC2[k2]>=2:
                                print('\tTRUE', file=singleC2outfile)
                            else:
                                print('\tFALSE', file=singleC2outfile)
            # print C2 doubles to a single C2 outfile
            # if naive library has at least 50 counts, and post-selection library has at least 2 counts
            with open('Doubles-C2-processed.txt', 'a') as doubleC2outfile:
                print('Pos1\tPos2\tMut1\tMut2\tWT_AA1\tWT_AA2\tcount_r1_t0\tcount_r1_t1\tis.fitness\tis.reads0\tfitness1\tfitness2\tsigma1\tsigma2\tfitness\tsigma', file=doubleC2outfile)
                for k,v in sorted(fitnessC2.items()):
                    for k2,v2 in sorted(doubles_sigmaC2.items()):
                        if k in k2:
                            print(str(WT1[1:]) + '\t' + str(WT2[1:]) + '\t' + str(k[0]) + '\t' + str(k[-1]) + '\t'
                                 + str(WT1[0]) + '\t' + str(WT2[0]) + '\t' + str(dNS[k]) + '\t' + str(dC2[k]) + '\t'
                                 + 'TRUE', file=doubleC2outfile, end='')
                            if dNS[k]>=50 and dC2[k]>=2:
                                print('\tTRUE', file=doubleC2outfile, end='')
                            else:
                                print('\tFALSE', file=doubleC2outfile, end='')
                            print('\t' + str(single_mutsC2[str(k[0] + ':' + WT2[0])]) + '\t' + str(single_mutsC2[str(WT1[0] + ':' + k[-1])])+ '\t'
                                  + str(singles_sigmaC2[str(k[0] + ':' + WT2[0])]) + '\t' + str(singles_sigmaC2[str(WT1[0] + ':' + k[-1])]) + '\t'
                                  + str(v) + '\t' + str(doubles_sigmaC2[str(k)]), file=doubleC2outfile)

In [6]:
counts_to_fitness('N104','Y105')